#### Solutions to the problems of Chapter 10
### Maps, Hash Tables, and Skip Lists

## Basic implementations and helper classes

In [56]:
from collections import abc
class MapBase(abc.MutableMapping):
    class _Item:
        __slots__='_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v
            
        def __eq__(self, other):
            return self._key == other._key 
        
        def __neq__(self, other):
            return not(self == other) 
        
        def __lt__(self, other):
            return self._key < other._key 

In [79]:
class UnsortedTableMap(MapBase):
    
    def __init__(self):
        self._table = [] 
        
    def __getitem__(self, k):
        for item in self._table:
            if k == item._key:
                return item._value 
            else:
                raise KeyError('Key Error: '+repr(k))
        
    def __setitem__(self, k, v):
        for item in self._table:
            if k == item._key:
                item._value = v 
                return
        self._table.append(self._Item(k, v))
                
    def __delitem__(self, k):
        for i in range(len(self._table)):
            if k == self._table[i]._key:
                self._table.pop(i)
                return 
        raise KeyError('Key Error: '+repr(k))
                
    def __len__(self): 
        return len(self._table)
    
    def __iter__(self): 
        for item in self._table:
            yield item._key 

In [65]:
# di = {}
# with open('raven', 'r') as f: 
#     pieces = f.read().lower().split()
#     for piece in pieces: 
#         word = ''.join(c for c in piece if c.isalpha())
#         di[word] = di.get(word, 0)+1 
    
    
# sorted_dict = sorted(di.items(), key= lambda x:x[1], reverse=True)
# sorted_dict